In [0]:
%pip install openai pandas openpyxl
%pip install python-docx

In [0]:
import os
import openai
import pandas as pd
import json
import openpyxl
import random

In [0]:
from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": "You are a helpful assistant whose role is to quality assure the content in the input data. Your tasks inclValidating accuracy of the provided information.Cross-checking consistency of the content with any tables, plots, or visual data representations given.Highlighting discrepancies or errors between the written content and the numerical/graphical data."
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)

In [0]:
# Load the Word file which is in the same folder
try:    import docx
except ModuleNotFoundError:
    print("The 'python-docx' package is not installed. Please install it in your environment.")

# Load the Word file which is in the same folder
doc = docx.Document("ecf_text.docx")


# Display the content of the Word file
for para in doc.paragraphs:
    print(para.text)

In [0]:
##checking for headlines: 
text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

# Initialize Databricks LLM client
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"   # replace with your Databricks endpoint
)

# Simple query to check if the LLM understands
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": f"Here is my document text:\n\n{text}\n\nCan you list any headlines you detect?"
        }
    ],
    max_tokens=500
)

doc_headlines = response.choices[0].message.content.split("\n")
doc_headlines = [h.strip() for h in doc_headlines if h.strip()]

In [0]:
print(doc_headlines)

In [0]:
import pandas as pd

# Load the CSV file
csv_df = pd.read_csv("teacher_leader_development_ecf_2024_25.csv")
meta_df =pd.read_csv("teacher_leader_development_ecf_2024_25.meta.csv")
mentors =pd.read_csv("teacher_leader_development_ecf_mentor_2024_25.csv")
mentors_meta = pd.read_csv("teacher_leader_development_ecf_mentor_2024_25.meta.csv")
retention = pd.read_csv("teacher_leader_development_ecf_retention_2024_25.csv")
retention_meta = pd.read_csv("teacher_leader_development_ecf_retention_2024_25.meta.csv")

In [0]:
##### now onto the csv files: only keeping the values which are relavnt for healdines:  
# #Function to filter a dataframe: keep 'time_period' plus columns from 'breakdown' to 'group_name'
def filter_df(df):
    if "breakdown" in df.columns and "time_period" in df.columns:
        start_idx = df.columns.get_loc("breakdown")
        # Keep all columns from "breakdown" to the end
        cols_to_keep = ["time_period"] + list(df.columns[start_idx:])
        return df[cols_to_keep]
    else:
        print("One or more required columns are missing.")
        return df


# Apply filtering
filtered_csv_df = filter_df(csv_df)
filtered_mentors = filter_df(mentors)
filtered_retention = filter_df(retention)

In [0]:
# Preview
print(filtered_csv_df.head())
print(filtered_mentors.head())
print(filtered_retention.head())

In [0]:
import re

def extract_numbers_from_df(df):
    """Extract all integers from all columns in a DataFrame."""
    numbers = set()
    for col in df.columns:
        # Convert everything to string to make regex work
        for val in df[col].astype(str):
            found = re.findall(r'\d+', val)
            numbers.update(map(int, found))
    return numbers

In [0]:
# Extract numbers from each filtered DataFrame
csv_numbers = extract_numbers_from_df(filtered_csv_df)
mentor_numbers = extract_numbers_from_df(filtered_mentors)
retention_numbers = extract_numbers_from_df(filtered_retention)

In [0]:
# Combine all CSV numbers into one list
all_csv_numbers = csv_numbers | mentor_numbers | retention_numbers

In [0]:
#### extract numbers from text data
import re

# Join all lines into one string
text = " ".join(doc_headlines)

# Regex to find numbers (integers, decimals, and numbers with commas)
all_numbers = re.findall(r'\d[\d,]*\.?\d*', text)

# Convert numbers to int or float
numbers = []
for num in all_numbers:
    # Ignore years like 2021-2024
    if re.match(r'^(20\d{2})$', num):
        continue
    # Ignore fiscal-year style numbers like 2023/24
    if re.match(r'^20\d{2}/\d{2}$', num):
        continue
    # Convert to int or float
    numbers.append(float(num.replace(',', '')) if '.' in num else int(num.replace(',', '')))

print(numbers)


In [0]:
diff_text_not_in_csv = [num for num in numbers if num not in all_csv_numbers]
diff_csv_not_in_text = [num for num in all_csv_numbers if num not in numbers]

print("Numbers in text but not in CSVs:", diff_text_not_in_csv)

In [0]:
###### OR ask LLM to do it: 

# Join all lines into one string
text = " ".join(doc_headlines)

# Regex to find numbers (integers, decimals, and numbers with commas)
all_numbers = re.findall(r'\d[\d,]*\.?\d*', text)

# Convert numbers to int or float
numbers = []
for num in all_numbers:
    # Ignore years like 2021-2024
    if re.match(r'^(20\d{2})$', num):
        continue
    # Ignore fiscal-year style numbers like 2023/24
    if re.match(r'^20\d{2}/\d{2}$', num):
        continue
    # Convert to int or float
    numbers.append(float(num.replace(',', '')) if '.' in num else int(num.replace(',', '')))

print(numbers)

In [0]:
from openai import OpenAI
import os

# How to get your Databricks token: https://docs.databricks.com/en/dev-tools/auth/pat.html
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[
        {
            "role": "user",
            "content": "You are a highly attentive assistant responsible for performing quality assurance on the provided text and table data. Your tasks are:Verify Figures: Check that all numerical values in the text match the corresponding figures in the table. Highlight any discrepancies.Validate Commentary: Ensure that the commentary is logical, accurate, and consistent with the table data. Identify any misleading, unclear, or contradictory statements.Clarity & Consistency: Check for grammar, spelling, and formatting errors that could affect readability or comprehension.Provide Feedback: List any issues found and suggest corrections or improvements"
        }
    ],
    max_tokens=5000
)

print(response.choices[0].message.content)

In [0]:
prompt = f"""
You are a highly attentive assistant responsible for performing quality assurance on the provided text and table data.

Text:
{doc_headlines}

Table:
{filtered_csv_df.to_csv(index=False)}

Your tasks:
1. Verify Figures: Check that all numerical values in the text match the corresponding figures in the table. Highlight any discrepancies.
2. Validate Commentary: Ensure that the commentary is logical, accurate, and consistent with the table data. Identify any misleading, unclear, or contradictory statements.
3. Clarity & Consistency: Check for grammar, spelling, and formatting errors that could affect readability or comprehension.
4. Provide Feedback: List any issues found and suggest corrections or improvements.

Please provide the output in JSON format with keys:
'figure_discrepancies', 'commentary_issues', 'grammar_issues', 'suggestions'
"""



In [0]:
# --------------------------
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

# --------------------------
# Step 3: Call the LLM
# --------------------------
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=5000
)

In [0]:
print(response.choices[0].message.content)

In [0]:
from IPython.display import Markdown, display

output_text = response.choices[0].message.content

# Display as Markdown
display(Markdown(output_text))


In [0]:
prompt = f"""
You are a highly attentive assistant responsible for performing quality assurance on the provided text and table data.

Text:
{doc_headlines}

Table:
{filtered_csv_df.to_csv(index=False)}

Your tasks:
1. Check if the figures in doc_headlines matches to the numbers in filtered_csv_df

Please provide the output in JSON format with keys:
'figure_discrepancies', 'commentary_issues', 'grammar_issues', 'suggestions'
"""

In [0]:
# --------------------------
DATABRICKS_TOKEN = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

client = OpenAI(
    api_key=DATABRICKS_TOKEN,
    base_url="https://adb-2220072380334347.7.azuredatabricks.net/serving-endpoints"
)

# --------------------------
# Step 3: Call the LLM
# --------------------------
response = client.chat.completions.create(
    model="databricks-claude-3-7-sonnet",
    messages=[{"role": "user", "content": prompt}],
    max_tokens=5000
)

In [0]:
print(response.choices[0].message.content)

In [0]:
from IPython.display import Markdown, display

In [0]:
from IPython.display import display, Markdown

# Correctly access the content attribute
display(Markdown(response.choices[0].message.content))